![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_RoBERTaForMultipleChoice.ipynb)

# Import OpenVINO RoBertaForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting RoBertaForMultipleChoice  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for RoBertaForMultipleChoice from RoBertaForMultipleChoice  and they have to be in `Multiple Choice` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- Normally, to load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. However, ForMultipleChoice is not yet available so we will use `openvino.convert_model()` after exporting ONNX model
- We'll use [SyedArsal/roberta-urdu-small-finetuned-news](https://huggingface.co/SyedArsal/roberta-urdu-small-finetuned-news) model from HuggingFace as an example
- We also need the `vocab.txt` saved from `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
!pip install --upgrade pip
!pip install -q --upgrade transformers[onnx] optimum openvino==2024.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 134.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 170.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 182.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.

In [ ]:
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "SyedArsal/roberta-urdu-small-finetuned-news"
ONNX_MODEL_PATH = f"onnx_models/roberta_multiple_choice"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.54M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
!ls -lh {ONNX_MODEL_PATH}

total 489M
-rw-r--r-- 1 root root  744 Jan  3 19:52 config.json
-rw-r--r-- 1 root root 1.1M Jan  3 19:52 merges.txt
-rw-r--r-- 1 root root 481M Jan  3 19:52 model.onnx
-rw-r--r-- 1 root root  957 Jan  3 19:52 special_tokens_map.json
-rw-r--r-- 1 root root 1.4K Jan  3 19:52 tokenizer_config.json
-rw-r--r-- 1 root root 4.9M Jan  3 19:52 tokenizer.json
-rw-r--r-- 1 root root 1.5M Jan  3 19:52 vocab.json


In [ ]:
import openvino as ov

# ov.convert_model returns an openvino.runtime.Model object
ov_model = ov.convert_model(f"{ONNX_MODEL_PATH}/model.onnx")

OPENVINO_MODEL_PATH = "./openvino_models"
# then model can be serialized to *.xml & *.bin files
ov.save_model(ov_model, f"{OPENVINO_MODEL_PATH}/openvino_model.xml")

In [ ]:
!ls -lh {OPENVINO_MODEL_PATH}

total 241M
-rw-r--r-- 1 root root 241M Jan  3 19:52 openvino_model.bin
-rw-r--r-- 1 root root 624K Jan  3 19:52 openvino_model.xml


In [ ]:
import json

# Read the vocab JSON file
with open('{}/vocab.json'.format(ONNX_MODEL_PATH), 'r') as json_file:
    tokenizer = json.load(json_file)

# let's save the vocab as txt file
with open('{}/vocab.txt'.format(ONNX_MODEL_PATH), 'w') as keys_file:
  for item in tokenizer.keys():
    keys_file.write("%s\n" % item)

In [ ]:
!mkdir {OPENVINO_MODEL_PATH}/assets

In [ ]:
!mv {ONNX_MODEL_PATH}/vocab.txt {OPENVINO_MODEL_PATH}/assets
!mv {ONNX_MODEL_PATH}/merges.txt {OPENVINO_MODEL_PATH}/assets

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -lR {OPENVINO_MODEL_PATH}

./openvino_models:
total 246688
drwxr-xr-x 2 root root      4096 Jan  3 19:52 assets
-rw-r--r-- 1 root root 251962160 Jan  3 19:52 openvino_model.bin
-rw-r--r-- 1 root root    638403 Jan  3 19:52 openvino_model.xml

./openvino_models/assets:
total 2200
-rw-r--r-- 1 root root 1150119 Jan  3 19:52 merges.txt
-rw-r--r-- 1 root root 1099068 Jan  3 19:52 vocab.txt


## Import and Save RoBertaForMultipleChoice in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.5.3


- Let's use `loadSavedModel` functon in `RoBertaForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForMultipleChoice` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *


roBertaMultpleChoiceClassifier = RoBertaForMultipleChoice.loadSavedModel(
     f"{OPENVINO_MODEL_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
roBertaMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL_PATH}
!rm -rf {OPENVINO_MODEL_PATH}

Awesome 😎  !

This is your `RoBertaForMultipleChoice` model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 246728
drwxr-xr-x 4 root root      4096 Jan  3 19:53 fields
drwxr-xr-x 2 root root      4096 Jan  3 19:53 metadata
-rw-r--r-- 1 root root 252639391 Jan  3 19:53 roberta_mc_classification_openvino


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `RoBertaForMultipleChoice` model 😊

In [ ]:
 testing_data = [
            ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
             "It is eaten with a fork and a knife, It is eaten while held in the hand."),

            ("The Eiffel Tower is located in which country?",
             "Germany, France, Italy"),

            ("Which animal is known as the king of the jungle?",
             "Lion, Elephant, Tiger, Leopard"),

            ("Water boils at what temperature?",
             "90°C, 120°C, 100°C"),

            ("Which planet is known as the Red Planet?",
             "Jupiter, Mars, Venus"),

            ("Which language is primarily spoken in Brazil?",
             "Spanish, Portuguese, English"),

            ("The Great Wall of China was built to protect against invasions from which group?",
             "The Greeks, The Romans, The Mongols, The Persians"),

            ("Which chemical element has the symbol 'O'?",
             "Oxygenm, Osmium, Ozone"),

            ("Which continent is the Sahara Desert located in?",
             "Asia, Africa, South America"),

            ("Which artist painted the Mona Lisa?",
             "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
        ]

In [ ]:
testing_df = spark.createDataFrame(testing_data, ["question", "choices"])
testing_df.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = MultiDocumentAssembler() \
            .setInputCols(["question", "choices"]) \
            .setOutputCols(["document_question", "document_choices"])

roberta_for_multiple_choice = RoBertaForMultipleChoice() \
  .load(f"{MODEL_NAME}_spark_nlp_openvino") \
  .setInputCols(["document_question", "document_choices"])\
  .setOutputCol("answer") \
  .setBatchSize(4)

pipeline = Pipeline(stages=[document_assembler, roberta_for_multiple_choice])
pipeline_model = pipeline.fit(testing_df)

pipeline_df = pipeline_model.transform(testing_df)

pipeline_df.select("answer").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------+
|answer                                                                                                     |
+-----------------------------------------------------------------------------------------------------------+
|[{chunk, 0, 34, It is eaten with a fork and a knife, {sentence -> 0, chunk -> 0, score -> 0.60569566}, []}]|
|[{chunk, 0, 6, Germany, {sentence -> 0, chunk -> 0, score -> 0.33706638}, []}]                             |
|[{chunk, 0, 5,  Tiger, {sentence -> 0, chunk -> 0, score -> 0.25371727}, []}]                              |
|[{chunk, 0, 3, 90°C, {sentence -> 0, chunk -> 0, score -> 0.336369}, []}]                                  |
|[{chunk, 0, 6, Jupiter, {sentence -> 0, chunk -> 0, score -> 0.37836587}, []}]                             |
|[{chunk, 0, 7,  English, {sentence -> 0, chunk -> 0, score -> 0.339204}, []}]                              |
|[{chunk, 

That's it! You can now go wild and use hundreds of `RoBertaForMultipleChoice` models from HuggingFace 🤗 in Spark NLP 🚀
